In [1]:
cik = 1137774
import requests
from bs4 import BeautifulSoup
import pandas as pd


def fs_page(cik):
    url="https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK={}&type=10-k&dateb=&owner=exclude&count=40".format(cik)   
    response=requests.get(url)
    result_page=BeautifulSoup(response.content,'lxml')
    exturl = result_page.find_all('a',id='documentsbutton')
    list_=list()
    for ext in exturl:
        link = "https://www.sec.gov/"+ ext.get('href')
        list_.append(link)
    dict_k = dict()

    for i in list_:
        page = BeautifulSoup(requests.get(i).content, 'lxml')
        if page.find('strong').text == "Form 10-K":
            year = page.find_all('div',class_="info")[3].text
            test="https://www.sec.gov/"+ page.find_all('td')[2].find('a').get('href')
            if "htm" in test:
        #         print(test)
                dict_k[year]=test
    return dict_k

In [47]:
def ConObligation_Number(cik,keyword="Payments Due by Period"):
    import time 
    start_time =time.time()
    dict_k=fs_page(cik)
    new_dict=dict()
    for key, value in dict_k.items():
        try:
            dfs = pd.read_html(value, match=keyword) 
            new_row = list()
            row= dfs[0]
            last = row.dropna(thresh=5).iloc[-1]
            for i in last:
                if bool(set(str(i)) & set('0123456789')):
                    new_row.append(float(str(i).strip('$').replace(',',''))) 
                else:
                    pass
            new_dict[key]=new_row[new_row.index(max(new_row))+1]
        except:
            new_dict[key]='N/A'
    print('---%s seconds---'%(time.time()-start_time))
    return cik,new_dict
    


In [68]:
#set the working directory
dict_key=pd.read_excel('keywords_Insurance.xlsx')
def get_keyword(cik):
    import pandas as pd
    dict_key=pd.read_excel('keywords_Insurance.xlsx')
    return str(dict_key['keywords'][dict_key['CIK']== int(cik)].iloc[0])

In [74]:
ConObligation_Number(5272,get_keyword(5272))

---569.078000069 seconds---


(5272,
 {u'2003-12-31': 27826.0,
  u'2004-12-31': 58271.0,
  u'2005-12-31': 67974.0,
  u'2006-12-31': 103120.0,
  u'2007-12-31': 120332.0,
  u'2008-12-31': 92614.0,
  u'2009-12-31': 68748.0,
  u'2010-12-31': 54604.0,
  u'2011-12-31': 48241.0,
  u'2012-12-31': 51299.0,
  u'2013-12-31': 47705.0,
  u'2014-12-31': 39169.0,
  u'2015-12-31': 37774.0,
  u'2016-12-31': 36834.0})

In [75]:
def keywordlist():
    ava_list=list()
    inava_list=list()
    for i in dict_key['CIK']:
        if get_keyword(i) != 'nan':
            ava_list.append((i,get_keyword(i)))
        else:
            inava_list.append(i)
    return ava_list,inava_list

In [73]:
ava_list,inava_list=keywordlist()
result=list()
for i in ava_list:
    temp=ConObligation_Number(i[0],i[1])
    result.append(temp)
    

[(874501, 'Payments Due by Period'),
 (4977, 'Distribution of Payments by Period'),
 (5272, 'Payments due by Period'),
 (1267238, 'contractual obligations'),
 (899051, 'contractual cash obligation'),
 (315293, 'Payments due in'),
 (11544, 'Estimated Payments By Periods'),
 (1067983, 'Estimated payments due by period'),
 (896159, 'Payments Due By Period'),
 (20286, 'Payments due by period'),
 (21175, 'contractual obligation'),
 (1054833, 'contractual obligation'),
 (1276520, 'Payments due by period'),
 (874766, 'Payments due by period'),
 (49071, 'Payments Due by Period'),
 (60086, 'Payments Due by Period'),
 (62709, 'Contractual Obligations'),
 (1126328, 'Payments due in year ending'),
 (80661, 'Contractual Obligations'),
 (86104, 'Payment By'),
 (320335, 'Actual\nActual Liability'),
 (86312, 'Payments Due by Period'),
 (731766, 'Total contractual obligations'),
 (5513, 'Payments Due'),
 (1156039, 'Payments Due by Period')]

In [46]:
df_final=pd.DataFrame(new_dict,index=(['Prudential'])) 
df_final

5272